In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import time
import pickle
from surprise import Dataset
from surprise.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

def load_movielens_data():
    """Load the MovieLens dataset and prepare it for training"""
    print("Loading MovieLens 100K dataset...")
    # Use the built-in movielens-100k dataset
    data = Dataset.load_builtin('ml-100k')
    
    # Split the data into train and test sets
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
    
    print(f"Dataset loaded. Training set size: {len(trainset.build_anti_testset())}, Test set size: {len(testset)}")
    
    # Convert to format suitable for TensorFlow
    raw_trainset = []
    for uid, iid, rating in trainset.all_ratings():
        raw_uid = trainset.to_raw_uid(uid)
        raw_iid = trainset.to_raw_iid(iid)
        raw_trainset.append((raw_uid, raw_iid, float(rating)))
    
    raw_testset = [(uid, iid, float(rating)) for uid, iid, rating in testset]
    
    # Create pandas DataFrames
    train_df = pd.DataFrame(raw_trainset, columns=['user_id', 'movie_id', 'rating'])
    test_df = pd.DataFrame(raw_testset, columns=['user_id', 'movie_id', 'rating'])
    
    # Extract unique users and movies
    unique_user_ids = sorted(list(set(train_df['user_id'].unique()).union(set(test_df['user_id'].unique()))))
    unique_movie_ids = sorted(list(set(train_df['movie_id'].unique()).union(set(test_df['movie_id'].unique()))))
    
    # Create vocabularies
    user_vocab = tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
    movie_vocab = tf.keras.layers.StringLookup(vocabulary=unique_movie_ids, mask_token=None)
    
    # Convert to TensorFlow datasets
    def df_to_tf_dataset(df, shuffle=True, batch_size=128):
        features = {
            "user_id": tf.cast(df["user_id"].values, tf.string),
            "movie_id": tf.cast(df["movie_id"].values, tf.string),
        }
        
        # Separate the targets (ratings)
        labels = tf.cast(df["rating"].values, tf.float32)
        
        dataset = tf.data.Dataset.from_tensor_slices((features, labels))
        
        if shuffle:
            dataset = dataset.shuffle(buffer_size=len(df))
            
        return dataset.batch(batch_size)
    
    train_dataset = df_to_tf_dataset(train_df)
    test_dataset = df_to_tf_dataset(test_df, shuffle=False)
    
    return train_dataset, test_dataset, user_vocab, movie_vocab, len(unique_user_ids), len(unique_movie_ids)

def build_ncf_model(user_vocab, movie_vocab, num_users, num_movies):
    """Build Neural Collaborative Filtering model"""
    class NCFModel(tf.keras.Model):
        def __init__(self, user_vocab, movie_vocab, num_users, num_movies):
            super().__init__()
            
            # User embedding
            self.user_lookup = user_vocab
            self.user_embedding = tf.keras.layers.Embedding(
                num_users + 1, 32, name="user_embedding")
            
            # Movie embedding
            self.movie_lookup = movie_vocab
            self.movie_embedding = tf.keras.layers.Embedding(
                num_movies + 1, 32, name="movie_embedding")
            
            # MLP layers
            self.dense_layers = [
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dense(16, activation='relu'),
            ]
            
            # Output layer
            self.rating_pred = tf.keras.layers.Dense(1)
            
        def call(self, inputs):
            # Get user and movie IDs
            user_id = inputs["user_id"]
            movie_id = inputs["movie_id"]
            
            # Look up embeddings
            user_embed = self.user_embedding(self.user_lookup(user_id))
            movie_embed = self.movie_embedding(self.movie_lookup(movie_id))
            
            # Concatenate embeddings
            concat = tf.concat([user_embed, movie_embed], axis=1)
            
            # Pass through dense layers
            x = concat
            for layer in self.dense_layers:
                x = layer(x)
                
            # Output prediction
            return self.rating_pred(x)
    
    # Create the model
    model = NCFModel(user_vocab, movie_vocab, num_users, num_movies)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.MeanSquaredError()
    )
    
    return model

def save_keras_model(model, name):
    """Save a Keras model properly using TF SavedModel format"""
    # Make sure the models directory exists
    if not os.path.exists('models'):
        os.makedirs('models')
    
    # Path for the saved model
    model_path = os.path.join('models', name)
    
    try:
        # Save the model using TensorFlow SavedModel format
        # This works for subclassed models
        model.save(model_path, save_format="tf")
        print(f"Model saved successfully to {model_path}")
        
        # Save model info
        model_info = {
            'model_type': 'Neural Collaborative Filtering',
            'model_path': model_path,
            'is_tensorflow': True,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
        }
        
        info_path = os.path.join('models', 'ncf_model_info.pkl')
        with open(info_path, 'wb') as f:
            pickle.dump(model_info, f)
        print(f"Model info saved to {info_path}")
        
        return True
    except Exception as e:
        print(f"Error saving model: {e}")
        
        # Try saving weights as a fallback
        try:
            weights_path = os.path.join('models', f"{name}_weights.h5")
            model.save_weights(weights_path)
            print(f"Saved model weights to {weights_path}")
            
            # Update the info file
            model_info = {
                'model_type': 'Neural Collaborative Filtering',
                'weights_path': weights_path,
                'is_weights_only': True,
                'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
            }
            
            info_path = os.path.join('models', 'ncf_model_info.pkl')
            with open(info_path, 'wb') as f:
                pickle.dump(model_info, f)
            
            return True
        except Exception as e2:
            print(f"Error saving weights: {e2}")
            return False

def train_and_save_ncf():
    """Train and save the Neural Collaborative Filtering model"""
    print("Starting Neural Collaborative Filtering model training...")
    
    # Load and prepare data
    train_dataset, test_dataset, user_vocab, movie_vocab, num_users, num_movies = load_movielens_data()
    
    # Build the model
    model = build_ncf_model(user_vocab, movie_vocab, num_users, num_movies)
    
    # Get a sample batch from the dataset to build the model
    for features_batch, _ in train_dataset.take(1):
        # This will build the model by running a forward pass
        _ = model(features_batch)
        break
    
    # Now we can call summary
    print("Model built successfully. Summary:")
    model.summary()
    
    # Train the model
    print("\nTraining the model...")
    start_time = time.time()
    
    history = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=20,  # Reduced epochs for quicker training
        verbose=1
    )
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")
    
    # Evaluate the model
    print("\nEvaluating the model...")
    evaluation = model.evaluate(test_dataset, return_dict=True)
    rmse = np.sqrt(evaluation['loss'])
    print(f"RMSE: {rmse:.4f}")
    
    # Save the model
    print("\nSaving the model...")
    if save_keras_model(model, "ncf_model"):
        print("Neural Collaborative Filtering model saved successfully")
    else:
        print("Failed to save the model")
    
    print("\nModel training and saving complete!")

if __name__ == "__main__":
    train_and_save_ncf()

2025-04-15 06:56:23.625211: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Starting Neural Collaborative Filtering model training...
Loading MovieLens 100K dataset...
Dataset loaded. Training set size: 1476893, Test set size: 20000


2025-04-15 06:56:25.817466: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2025-04-15 06:56:25.921443: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [80000]
	 [[{{node Placeholder/_2}}]]
2025-04-15 06:56:25.921669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [80000]
	 [[{{node Placeholder/_2}}]]


Model built successfully. Summary:
Model: "ncf_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 string_lookup (StringLookup  multiple                 0         
 )                                                               
                                                                 
 user_embedding (Embedding)  multiple                  30208     
                                                                 
 string_lookup_1 (StringLook  multiple                 0         
 up)                                                             
                                                                 
 movie_embedding (Embedding)  multiple                 53856     
                                                                 
 dense (Dense)               multiple                  4160      
                                                                 
 dense_1 (Dense)      

2025-04-15 06:56:26.132715: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [80000]
	 [[{{node Placeholder/_0}}]]
2025-04-15 06:56:26.132919: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [80000]
	 [[{{node Placeholder/_2}}]]


623/625 [============================>.] - ETA: 0s - loss: 2.0109

2025-04-15 06:56:30.066171: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [20000]
	 [[{{node Placeholder/_2}}]]


625/625 [==============================] - 4s 5ms/step - loss: 2.0074 - val_loss: 0.9159
Epoch 2/20
625/625 [==============================] - 2s 4ms/step - loss: 0.8949 - val_loss: 0.8977
Epoch 3/20
625/625 [==============================] - 3s 4ms/step - loss: 0.8749 - val_loss: 0.8970
Epoch 4/20
625/625 [==============================] - 4s 5ms/step - loss: 0.8537 - val_loss: 0.8657
Epoch 5/20
625/625 [==============================] - 3s 5ms/step - loss: 0.8325 - val_loss: 0.8608
Epoch 6/20
625/625 [==============================] - 3s 4ms/step - loss: 0.8165 - val_loss: 0.8661
Epoch 7/20
625/625 [==============================] - 4s 6ms/step - loss: 0.7967 - val_loss: 0.8550
Epoch 8/20
625/625 [==============================] - 4s 6ms/step - loss: 0.7754 - val_loss: 0.8505
Epoch 9/20
625/625 [==============================] - 3s 4ms/step - loss: 0.7541 - val_loss: 0.8509
Epoch 10/20
625/625 [==============================] - 3s 4ms/step - loss: 0.7323 - val_loss: 0.8594
Epoch 11/2

Error saving model: this __dict__ descriptor does not support '_DictWrapper' objects
Saved model weights to models/ncf_model_weights.h5
Neural Collaborative Filtering model saved successfully

Model training and saving complete!
